In [2]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append('D:/Documents/these_pablo/Models/BEACH2016/Analysis')
print(os.getcwd())

C:\Users\pablo\Documents\pablo-models\pesti-beach16\calib_v1\mVar_v1


In [3]:
def getSoilDataCal(path, name):
    path += name + ".tss"
    obs = pd.read_table(path)
    return obs

# Function to add column ID, with first letter capitalized.
def newlabel(row, plot):
    return plot.capitalize() + '-' + str(int(row['Jdays']))

In [4]:
path = 'D:/Documents/these_pablo/Models/BEACH2016/Analysis/Data/BEACH_R/'
path = 'C:/Users/pablo\Documents/pablo-models/pesti-beach16/Analysis/Data/BEACH_R/'


In [5]:
def get_dataframe(name_obs_detail, name_obs_comp, name_sim):
    """
    name_obs_detail = 'delta_det_cal' | 'conc_det_cal'
    name_obs_comp = 'delta_comp_cal' | 'conc_comp_cal'
    name_sim = 'd13C_real' | 'CONC_real'
    
    """
    
    detail_plots = ['n1', 'n2', 'n3', 'n4', 'n5', 'n7', 'n8',
                    'v4', 'v5', 'v7', 'v8', 'v9', 'v10',
                    's11', 's12', 's13']
    
    matches = []
    for sample in range(len(detail_plots)):
        plot = detail_plots[sample]

        # Simulated
        filename = "resM_" + plot + name_sim + ".tss"  
        det_sim = pd.read_table(filename, 
                                    skiprows=4, delim_whitespace=True,
                                    names=['Jdays', 'SIM'],
                                    header=None)
        # Observed 
        det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, plot),axis=1)  # Add ID
        n = getSoilDataCal(path, name_obs_detail)   # Name observed detailed

        match = pd.merge(n, det_sim, how='inner', on='IDcal')
        matches.append(match)
        
    conc_det_dat = pd.concat(matches)
    
    # Prepare composites and concatenate
    sample_comp = ['nor', 'val', 'sou']
    matches = []

    for sample in range(len(sample_comp)):
        plot = sample_comp[sample]
        filename = "resM_" + plot + name_sim + ".tss"
        det_sim = pd.read_table(filename, 
                                    skiprows=4, delim_whitespace=True,
                                    names=['Jdays', 'SIM'],
                                    header=None)
        label = plot[0]
        det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, label),axis=1) 
        n = getSoilDataCal(path, name_obs_comp)  # Name observed composites
        match = pd.merge(n, det_sim, how='inner', on='IDcal')
        matches.append(match)
    # Concatenate composites
    conc_comp_dat = pd.concat(matches)
    conc_dat = pd.concat([conc_det_dat, conc_comp_dat])
    
    # Concatenate detailed AND COMPOSITES
    return conc_dat

In [6]:
def get_error(var):
    
    # Define observation dataframes to use
    if var == "ug.g":
        name_obs_detail = 'conc_det_cal'
        name_obs_comp = 'conc_comp_cal'
        name_sim = 'CONC_real'
    else:
        name_obs_detail = 'delta_det_cal'
        name_obs_comp = 'delta_comp_cal'
        name_sim = 'd13C_real'
        
    df1 = get_dataframe(name_obs_detail, name_obs_comp, name_sim)
    
    # Nash
    mean = df1[var].mean()
    # Diff sim vs. obs
    dfn = df1.assign(diff_sim=lambda row: (row['SIM'] - row[var]) ** 2)
    err_sim = dfn['diff_sim'].sum()
    # Variance
    dfn = dfn.assign(diff_obs=lambda row: (row[var] - mean) ** 2)
    err_obs = dfn['diff_obs'].sum()
    error = err_sim / err_obs
    # nash = 1 - error
    if var == 'ug.g':  # Log only for concentrations
        lnmean = np.log(df1[var]).mean()
        # Log Diff sim vs. obs
        dfn = dfn.assign(lndiff_sim=lambda row: (np.log(row['SIM']) - np.log(row[var])) ** 2)
        err_lnsim = dfn['lndiff_sim'].sum()
        # Log variance
        dfn = dfn.assign(lndiff_obs=lambda row: (np.log(row[var]) - lnmean) ** 2)
        err_lnobs = dfn['lndiff_obs'].sum()
        error += err_lnsim / err_lnobs
        error *= 0.5
    return error

In [7]:
get_error('d13C')

0.5550627880917681

In [8]:
get_error('ug.g')

0.16420873034135822

In [9]:
nash = 1 - 0.5*(get_error('d13C') + get_error('ug.g'))
nash

0.6403642407834369